In [145]:
import xml
import xml.etree.ElementTree as ET
import os
from os import path
import glob
from collections import defaultdict

In [139]:
data_dir = "/home/shtoshni/Research/events/data/red/data/source"
source_files = glob.glob("{}/*/*".format(data_dir))

ann_dir = "/home/shtoshni/Research/events/data/red/data/annotation"
ann_files = glob.glob("{}/*/*".format(ann_dir))

In [140]:
event_counter = 0
entity_counter = 0
for source_file in source_files:
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")

    tree = ET.parse(ann_file)
    root = tree.getroot()
    for elem in root.iter('entity'):
    #     print(dir(elem))
        for sub_elem in elem.iter('type'):
            if sub_elem.text == 'EVENT':
                event_counter += 1
            elif sub_elem.text == 'ENTITY':
                entity_counter += 1
    
print(event_counter)
print(entity_counter)
                

8731
10319


In [141]:
source_file = "/home/shtoshni/Research/events/data/red/data/source/deft/04debcc4da342dc971bdef4210fe468a.mpdf"
source_str  = "".join(open(source_file).readlines())

base_name = path.basename(source_file)
dir_name = path.basename(path.dirname(source_file))

ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
tree = ET.parse(ann_file)
root = tree.getroot()

cluster_idx = 1
entity_id_to_cluster_id = {}
for elem in root.iter('relation'):
#     print(dir(elem))
    elem_type = list(elem.iter('type'))[0].text
    if elem_type == 'IDENTICAL':
        elem_props = list(elem.iter('properties'))[0]
        for sub_elem in elem_props.iter():
            if sub_elem.tag == 'FirstInstance' or sub_elem.tag == 'Coreferring_String':
                entity_id_to_cluster_id[sub_elem.text] = cluster_idx
            
        cluster_idx += 1

In [142]:
output_dir = "/home/shtoshni/Research/events/data/red/red_html"


HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head><body>'

entity_tag = '<div style="border:2px; display:inline; border-style: solid; border-color: red; padding: 10px; padding-right: 3px; padding-left: 3px">'
event_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: green; padding:10px; padding-right: 3px; padding-left: 3px">'
time_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: blue; padding:10px; padding-right: 3px; padding-left: 3px">'
doctime_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: orange; padding:10px; padding-right: 3px; padding-left: 3px">'
sectiontime_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: turquoise; padding:10px; padding-right: 3px; padding-left: 3px">'
duplicate_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: brown; padding:10px; padding-right: 3px; padding-left: 3px">'


end_tag = '</div>'

largest_padding = 11
padding_reduction = 3

In [143]:
for source_file in source_files:#["/home/shtoshni/Research/events/data/red/data/source/deft/04debcc4da342dc971bdef4210fe468a.mpdf"]:
    # Read the source doc
    source_lines = open(source_file).readlines()
    source_str = "".join(source_lines)
    
    # Read the annotation file
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
    
    tree = ET.parse(ann_file)
    root = tree.getroot()
    entity_list = []
    html_tag_list = defaultdict(list)
    
    # Get all the entity info
    for elem in root.iter('entity'):
        span_str = list(elem.iter('span'))[0].text
        span_start, span_end = [int(endpoint) for endpoint in span_str.split(",")]
        elem_id = list(elem.iter('id'))[0].text
        elem_type = list(elem.iter('type'))[0].text
        
        entity_list.append((elem_id, elem_type, (span_start, span_end)))
        
        if elem_type == 'ENTITY':
            html_tag = entity_tag
        elif elem_type == 'EVENT':
            html_tag = event_tag
        elif elem_type == 'TIMEX3':
            html_tag = time_tag
        elif elem_type == 'DOCTIME':
            html_tag = doctime_tag
        elif elem_type == 'SECTIONTIME':
            html_tag = sectiontime_tag
        elif elem_type == 'DUPLICATE':
            html_tag = duplicate_tag
        else:
            print("Sweet Glory! {} {}".format(elem_type, source_file))
        
        html_tag_list[span_start].append((html_tag, 'start', elem_id))
        html_tag_list[span_end].append((end_tag, 'end', elem_id))
    
    entity_list = sorted(entity_list, key=lambda x: x[2][0])
    for key in html_tag_list:
        # Sort all the HTML tags so that the end tags occur before the start tags
        html_tag_list[key] = sorted(html_tag_list[key], key=lambda x: x[1])

    # Get all the relationship info
    cluster_idx = 1
    entity_id_to_cluster_id = {}
    for elem in root.iter('relation'):
        elem_type = list(elem.iter('type'))[0].text
        if elem_type == 'IDENTICAL':
            elem_props = list(elem.iter('properties'))[0]
            for sub_elem in elem_props.iter():
                if sub_elem.tag == 'FirstInstance' or sub_elem.tag == 'Coreferring_String':
                    entity_id_to_cluster_id[sub_elem.text] = cluster_idx

            cluster_idx += 1
    
    
    html_string = HTML_START + '<div style="line-height: 3">'
    
    offset = 0 
    counter = 0
    source_str = source_str.replace("<", "~")
    source_str = source_str.replace(">", "^")
    for idx, token in enumerate(source_str):
        if idx in html_tag_list:
            tags = html_tag_list[idx]
            for tag, tag_type, elem_id in tags:
                if elem_id in entity_id_to_cluster_id and tag_type == 'end':
                    html_string += "<sub>" + str(entity_id_to_cluster_id[elem_id]) + "</sub>" 
                html_string += tag
                
        
        html_string += token
            
        
    html_string += "</div></body></html>"
    html_string = html_string.replace("\n", "\n<br/>")
    html_string = html_string.replace("~", "&lt;")
    html_string = html_string.replace("^", "&gt;")
    with open(path.join(output_dir, base_name + ".html"), "w") as f:
        f.write(html_string)

In [144]:
index_html = HTML_START + '<ol type="1">'

for source_file in source_files:
    base_file = path.basename(source_file)
    output_file = base_file + ".html"
    
    index_html += '<li> <a href="{}", target="_blank">'.format(output_file) + base_file + '</a></li>\n'
    

index_html += '</ol>\n</body>\n</html>'
with open(path.join(output_dir, "index.html"), "w") as g:
    g.write(index_html)